## 1. Keyword/Text search with Standard Analyzer

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

https://iksch.search.windows.net


In [4]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="title", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="body", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="date", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=False, filterable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True,
                        facetable=True, filterable=True),
        SearchableField(name="docid", type=SearchFieldDataType.String)
    ]

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "ncevent-std-index"
create_search_index(index_name)

 ncevent-std-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_json("./sample_events.json")
df.head()

,id,title,date,body
0,520783,"설날 기념 마이타임, 핫타임 이벤트",2025-01-24 12:00:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n설날 기념 마이타임...
1,520257,설 기념 깜짝 우편 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n2025년 설 연휴...
2,520255,"도노반, 에스텔라 전용 이벤트",2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n독립된 또 다른 세...
3,520254,특별한 새해 떡국 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n두 번째 설을 기념...
4,520253,+9 밤까마귀 장비 제작 이벤트,2025-01-22 22:10:00,\n자유를 향한 검은 날개\n안녕하세요. 나이트 크로우입니다.\n+9 밤까마귀 장비...


In [9]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'title': row['title'], 
                     'body': row['body'],
                     'date': row['date'],
                     'type': "event",
                     "docid": str(row["id"])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:02<00:00,  2.97it/s]


In [11]:
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

import json

def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "date", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}) {result['@search.score']:.10f}: {result['docid']}, {result['title']}, {result['date']}")  

## Demo

- basic keyword search
- filter
- paging
- faceting

### basic search

In [12]:
results = azsch_text_query('산코나')

Search Results:
1) 4.8371290000: 214626, 던전 이벤트, 2023-12-14T03:30:00Z
2) 4.6867366000: 474792, 던전 이벤트, 2024-11-07T09:52:00Z
3) 4.5582650000: 439637, 던전 이벤트, 2024-09-11T19:30:00Z
4) 4.5170830000: 264647, 출석 이벤트, 2024-03-27T18:30:00Z
5) 4.4692607000: 336151, 던전 이벤트, 2024-05-15T18:00:00Z
6) 4.3869390000: 344072, 출석 이벤트, 2024-05-22T18:30:00Z
7) 4.2305055000: 224612, 던전 이벤트, 2024-01-24T18:30:00Z
8) 4.1429367000: 234729, 던전 이벤트, 2024-03-06T18:00:00Z
9) 3.9897876000: 376712, 던전 이벤트, 2024-06-26T18:30:00Z
10) 3.7553036000: 406041, 던전 이벤트, 2024-08-07T18:30:00Z


In [13]:
results = azsch_text_query('산코나유적')

Search Results:


In [14]:
results = azsch_text_query('산코나 유적')

Search Results:
1) 9.0341660000: 264647, 출석 이벤트, 2024-03-27T18:30:00Z
2) 8.7738780000: 344072, 출석 이벤트, 2024-05-22T18:30:00Z
3) 7.9611910000: 234729, 던전 이벤트, 2024-03-06T18:00:00Z
4) 6.6323414000: 328844, 출석 이벤트, 2024-05-08T18:00:00Z
5) 6.2928300000: 336152, 부트캠프 전용 이벤트, 2024-05-15T18:00:00Z
6) 5.2764997000: 497726, 던전 이벤트, 2024-12-18T19:00:00Z
7) 5.0870824000: 219799, 푸른 용의 조각 이벤트, 2024-01-04T03:10:00Z
8) 4.8371290000: 214626, 던전 이벤트, 2023-12-14T03:30:00Z
9) 4.6867366000: 474792, 던전 이벤트, 2024-11-07T09:52:00Z
10) 4.6788406000: 239434, 출석 이벤트, 2024-03-13T18:35:00Z


### Hightlight

In [15]:
def azsch_text_query_highlight(query):

    results = search_client.search(  
        search_text=query,
        query_type=QueryType.Simple,
        search_mode=SearchMode.Any,
        search_fields=["body", "title"],
        select=["docid", "body", "title", "date", "type"],
        highlight_fields="body",
        query_language="ko-kr",
        top=5 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1):  
        if (result["@search.highlights"]):
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']}")
            for highlight in result['@search.highlights']['body']:
                print(f"    {highlight}")
        else:
            print(f"{result['docid']}: {result['@search.score']:.10f}: {result['title']} - {result['body']}")  

In [18]:
azsch_text_query_highlight("산코나유적")

Search Results:


In [19]:
azsch_text_query_highlight("산코나 유적")

Search Results:
264647: 9.0341660000: 출석 이벤트
    .* 게임 화면 상단 [이벤트] 아이콘을 통해 확인할 수 있습니다.불멸의 해방일지 II 출석 보상
분류보상 아이템수량1일차해방의 씨앗(귀속)
102일차[이벤트] 시간 충전 장치 - <em>산코나</em> <em>유적</em>(귀속)
23일차해방의 결정(귀속)
34일차해방의 씨앗(귀속)
105일차[이벤트] 시간 충전 장치 - <em>산코나</em> <em>유적</em>(귀속)
26일차해방의 결정(귀속)
37일차해방의 씨앗(귀속)
20* [이벤트] 아이템은 4월 18일(목) 08:00까지 사용할 수 있습니다.
344072: 8.7738780000: 출석 이벤트
    이벤트 기간: 5월 23일(목) 업데이트 후 ~ 6월 5일(수) 08:00까지이벤트 내용: 게임에 접속만 해도 성장에 도움이 되는 다양한 아이템을 획득할 수 있습니다.나이트 크로우 7일 출석 보상
분류보상 아이템수량1일차해방의 씨앗(귀속)1002일차해방의 결정(귀속)153일차[이벤트] 시간 충전 장치 -<em>산코나</em> <em>유적</em>(귀속)14일차석양의 찬란한 탈것/무기 외형 11회 선택 상자(귀속)15일차[이벤트] 시간 충전 장치 -<em>산코나</em> <em>유적</em>(귀속)16일차해방의 결정(귀속)157일차지혜의 원천(귀속)1
※ 이벤트 유의사항1.
234729: 7.9611910000: 던전 이벤트
    이벤트 기간 동안, <em>산코나</em> 유적에서 더 많은 경험치와 무기 숙련도를 얻을 수 있게 되었습니다.
    지금 이 순간, <em>산코나</em> 유적에서 누구보다 값진 성장을 향해 여러분의 여정을 시작하세요. 
    ■ 던전 이벤트
• <em>산코나</em> <em>유적</em> 경험치 및 무기 숙련도 효율 UP!
    • 이벤트 기간: 3월 7일(목) 업데이트 후 ~ 3월 21일(목) 업데이트 전까지• 이벤트 내용: 이벤트 기간 동안 <em>산코나</em> 유적의 경험치